<a href="https://colab.research.google.com/github/mahan2210/pytorch_practice/blob/main/Hyper_parameters_on_FashionMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

100%|██████████| 26421880/26421880 [00:08<00:00, 3129872.80it/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 276913.21it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 2731697.30it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 5347270.18it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [3]:
##Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [5]:
# prompt: define ADAM

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.308246  [    0/60000]
loss: 1.640470  [ 6400/60000]
loss: 1.602547  [12800/60000]
loss: 1.698169  [19200/60000]
loss: 1.715338  [25600/60000]
loss: 1.437456  [32000/60000]
loss: 1.387707  [38400/60000]
loss: 1.497806  [44800/60000]
loss: 1.500485  [51200/60000]
loss: 1.338387  [57600/60000]
Test Error: 
 Accuracy: 38.8%, Avg loss: 0.024047 

Epoch 2
-------------------------------
loss: 1.394154  [    0/60000]
loss: 1.484221  [ 6400/60000]
loss: 1.525491  [12800/60000]
loss: 1.606215  [19200/60000]
loss: 1.678607  [25600/60000]
loss: 1.388179  [32000/60000]
loss: 1.352727  [38400/60000]
loss: 1.483843  [44800/60000]
loss: 1.440135  [51200/60000]
loss: 1.247280  [57600/60000]
Test Error: 
 Accuracy: 40.0%, Avg loss: 0.023436 

Epoch 3
-------------------------------
loss: 1.361875  [    0/60000]
loss: 1.456051  [ 6400/60000]
loss: 1.479253  [12800/60000]
loss: 1.526226  [19200/60000]
loss: 1.651388  [25600/60000]
loss: 1.359647  [32000/600

In [8]:
torch.save(model.state_dict(), "data/Hyper-parameters on FashionMNIST.pth")

print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth
